In [2]:
 !pip install pyspark
 !pip install pyspark findspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=4dc0ac0cc02d0eeb3b77ba34ced0a1d5747f07f60b743ef8fdc28c55f7f59df2
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.functions import substring
from pyspark.sql.functions import concat
from pyspark.sql.functions import lit
from pyspark.sql.functions import col
from pyspark.sql.functions import trim
from pyspark.sql.functions import when
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import count


import datetime
import numpy as np
import pandas as pd

In [4]:
import findspark
findspark.init()

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder \
    .appName("CPS Analysis") \
    .getOrCreate()

# Load the data
data_path = "/content/drive/MyDrive/dec17pub.dat"
df = spark.read.format('csv').load(data_path, header = None)

#df = spark.read.option("delimiter", " ").option("header", "false").csv(data_path)

# Display the schema
df.printSchema()


root
 |-- _c0: string (nullable = true)



In [20]:
mapped_df = df.select(trim(substring(df._c0, 1, 15)).alias('full_household_id'), \
               trim(concat(substring(df._c0, 18, 4), lit("/"), substring(df._c0, 16, 2))).alias('time_of_interview'), \
               trim(substring(df._c0, 24, 3)).alias('final_outcome'), \
               trim(substring(df._c0, 31, 2)).alias('housing_unit_type'), \
               trim(substring(df._c0, 61, 2)).alias('household_type'), \
               trim(substring(df._c0, 33, 2)).alias('telephone'), \
               regexp_replace(trim(substring(df._c0, 35, 2)), "-", "").alias('access_phone_elsewhere'), \
               regexp_replace(trim(substring(df._c0, 37, 2)), "-", "").alias('telephone_interview_acceptable'), \
               regexp_replace(trim(substring(df._c0, 65, 2)), "-", "").alias('interview_type'), \
               regexp_replace(trim(substring(df._c0, 39, 2)), "-", "").alias('family_income_range'), \
               trim(substring(df._c0, 91, 1)).alias('geo_division'), \
               trim(substring(df._c0, 139, 2)).alias('race'))

In [21]:
mapped_df.show()

+-----------------+-----------------+-------------+-----------------+--------------+---------+----------------------+------------------------------+--------------+-------------------+------------+----+
|full_household_id|time_of_interview|final_outcome|housing_unit_type|household_type|telephone|access_phone_elsewhere|telephone_interview_acceptable|interview_type|family_income_range|geo_division|race|
+-----------------+-----------------+-------------+-----------------+--------------+---------+----------------------+------------------------------+--------------+-------------------+------------+----+
|  000004795110719|          2017/12|          201|                1|             1|        1|                     1|                             1|             2|                  9|           6|   1|
|  000004795110719|          2017/12|          201|                1|             1|        1|                     1|                             1|             2|                  9|         

In [22]:
final_mapped_df = mapped_df.select(mapped_df.full_household_id, \
                mapped_df.time_of_interview, \

                 when(mapped_df['final_outcome'] == lit("1"), lit("FULLY COMPLETE CATI INTERVIEW")). \
                when(mapped_df['final_outcome'] == lit("2"), lit("PARTIALLY COMPLETED CATI INTERVIEW")). \
                when(mapped_df['final_outcome'] == lit("3"), lit("COMPLETE BUT PERSONAL VISIT REQUESTED NEXT MONTH")). \
                when(mapped_df['final_outcome'] == lit("4"), lit("PARTIAL, NOT COMPLETE AT CLOSEOUT")). \
                when(mapped_df['final_outcome'] == lit("5"), lit("LABOR FORCE COMPLETE, SUPPLEMENT INCOMPLETE - CATI")). \
                when(mapped_df['final_outcome'] == lit("6"), lit("LF COMPLETE, SUPPLEMENT DK ITEMS INCOMPLETE AT CLOSEOUT–ASEC ONLY")). \
                when(mapped_df['final_outcome'] == lit("20"), lit("HH OCCUPIED ENTIRELY BY ARMED FORCES MEMBERS OR ALL UNDER 15 YEARS OF AGE")). \
                when(mapped_df['final_outcome'] == lit("201"), lit("CAPI COMPLETE")). \
                when(mapped_df['final_outcome'] == lit("202"), lit("CALLBACK NEEDED")). \
                when(mapped_df['final_outcome'] == lit("203"), lit("SUFFICIENT PARTIAL - PRECLOSEOUT")). \
                when(mapped_df['final_outcome'] == lit("204"), lit("SUFFICIENT PARTIAL - AT CLOSEOUT")). \
                when(mapped_df['final_outcome'] == lit("205"), lit("LABOR FORCE COMPLETE, - SUPPL. INCOMPLETE - CAPI")). \
                when(mapped_df['final_outcome'] == lit("213"), lit("LANGUAGE BARRIER")). \
                when(mapped_df['final_outcome'] == lit("214"), lit("UNABLE TO LOCATE")). \
                when(mapped_df['final_outcome'] == lit("216"), lit("NO ONE HOME")). \
                when(mapped_df['final_outcome'] == lit("217"), lit("TEMPORARILY ABSENT")). \
                when(mapped_df['final_outcome'] == lit("218"), lit("REFUSED")). \
                when(mapped_df['final_outcome'] == lit("219"), lit("OTHER OCCUPIED - SPECIFY")). \
                when(mapped_df['final_outcome'] == lit("223"), lit("ENTIRE HOUSEHOLD ARMED FORCES")). \
                when(mapped_df['final_outcome'] == lit("224"), lit("ENTIRE HOUSEHOLD UNDER 15")). \
                when(mapped_df['final_outcome'] == lit("225"), lit("TEMP. OCCUPIED W/PERSONS WITH URE")). \
                when(mapped_df['final_outcome'] == lit("226"), lit("VACANT REGULAR")). \
                when(mapped_df['final_outcome'] == lit("227"), lit("VACANT - STORAGE OF HHLD FURNITURE")). \
                when(mapped_df['final_outcome'] == lit("228"), lit("UNFIT, TO BE DEMOLISHED")). \
                when(mapped_df['final_outcome'] == lit("229"), lit("UNDER CONSTRUCTION, NOT READY")). \
                when(mapped_df['final_outcome'] == lit("230"), lit("CONVERTED TO TEMP BUSINESS OR STORAGE")). \
                when(mapped_df['final_outcome'] == lit("231"), lit("UNOCCUPIED TENT OR TRAILER SITE")). \
                when(mapped_df['final_outcome'] == lit("232"), lit("PERMIT GRANTED - CONSTRUCTION NOT STARTED")). \
                when(mapped_df['final_outcome'] == lit("233"), lit("OTHER - SPECIFY")). \
                when(mapped_df['final_outcome'] == lit("240"), lit("DEMOLISHED")). \
                when(mapped_df['final_outcome'] == lit("241"), lit("HOUSE OR TRAILER MOVED")). \
                when(mapped_df['final_outcome'] == lit("242"), lit("OUTSIDE SEGMENT")). \
                when(mapped_df['final_outcome'] == lit("243"), lit("CONVERTED TO PERM. BUSINESS OR STORAGE")). \
                when(mapped_df['final_outcome'] == lit("244"), lit("MERGED")). \
                when(mapped_df['final_outcome'] == lit("245"), lit("CONDEMNED")). \
                when(mapped_df['final_outcome'] == lit("246"), lit("BUILT AFTER APRIL 1, 2000")). \
                when(mapped_df['final_outcome'] == lit("247"), lit("UNUSED SERIAL NO./LISTING SHEET LINE")). \
                when(mapped_df['final_outcome'] == lit("248"), lit("OTHER - SPECIFY")). \
                when(mapped_df['final_outcome'] == lit("256"), lit("REMOVED DURING SUB-SAMPLING")). \
                when(mapped_df['final_outcome'] == lit("257"), lit("UNIT ALREADY HAD A CHANCE OF SELECTION")). \
                 otherwise('UNDEFINED').alias('final_outcome'), \
                when(mapped_df.housing_unit_type == 1, lit("HOUSE, APARTMENT, FLAT")). \
                 when(mapped_df.housing_unit_type == 2, lit("HU IN NONTRANSIENT HOTEL, MOTEL, ETC.")). \
                when(mapped_df.housing_unit_type == 3, lit("HU PERMANENT IN TRANSIENT HOTEL, MOTEL")). \
                when(mapped_df.housing_unit_type == 4, lit("HU IN ROOMING HOUSE")). \
                when(mapped_df.housing_unit_type == 5, lit("MOBILE HOME OR TRAILER W/NO PERM. ROOM ADDED")). \
                when(mapped_df.housing_unit_type == 6, lit("MOBILE HOME OR TRAILER W/1 OR MORE PERM. ROOMS ADDED")). \
                when(mapped_df.housing_unit_type == 7, lit("HU NOT SPECIFIED ABOVE")). \
                when(mapped_df.housing_unit_type == 8, lit("QUARTERS NOT HU IN ROOMING OR BRDING HS")). \
                when(mapped_df.housing_unit_type == 9, lit("UNIT NOT PERM. IN TRANSIENT HOTL, MOTL")). \
                when(mapped_df.housing_unit_type == 10, lit("UNOCCUPIED TENT SITE OR TRLR SITE")). \
                when(mapped_df.housing_unit_type == 11, lit("STUDENT QUARTERS IN COLLEGE DORM")). \
                when(mapped_df.housing_unit_type == 12, lit("OTHER UNIT NOT SPECIFIED ABOVE")). \
                 otherwise('UNDEFINED').alias('housing_unit_type'), \
                when(mapped_df.household_type == 1, lit("HUSBAND/WIFE PRIMARY FAMILY (NEITHER AF)")). \
                when(mapped_df.household_type == 2, lit("HUSB/WIFE PRIM. FAMILY (EITHER/BOTH AF)")). \
                when(mapped_df.household_type == 3, lit("UNMARRIED CIVILIAN MALE-PRIM. FAM HHLDER")). \
                when(mapped_df.household_type == 4, lit("UNMARRIED CIV. FEMALE-PRIM FAM HHLDER")). \
                when(mapped_df.household_type == 5, lit("PRIMARY FAMILY HHLDER-RP IN AF, UNMAR.")). \
                when(mapped_df.household_type == 6, lit("CIVILIAN MALE PRIMARY INDIVIDUAL")). \
                when(mapped_df.household_type == 7, lit("CIVILIAN FEMALE PRIMARY INDIVIDUAL")). \
                when(mapped_df.household_type == 8, lit("PRIMARY INDIVIDUAL HHLD-RP IN AF")). \
                when(mapped_df.household_type == 9, lit("GROUP QUARTERS WITH FAMILY")). \
                when(mapped_df.household_type == 10, lit("GROUP QUARTERS WITHOUT FAMILY")). \
                 otherwise('UNDEFINED').alias('household_type'), \
                when(mapped_df['telephone'] == 1, lit("YES")). \
                when(mapped_df['telephone'] == 2, lit("NO")). \
                 otherwise('UNDEFINED').alias('telephone'), \
                when(mapped_df['access_phone_elsewhere'] == lit("1"), lit("YES")). \
                when(mapped_df['access_phone_elsewhere'] == lit("2"), \
                lit("NO")).alias('access_phone_elsewhere'), \
                when(mapped_df['telephone_interview_acceptable'] == lit("1"), lit("YES")). \
                when(mapped_df['telephone_interview_acceptable'] == lit("2"), \
                lit("NO")).otherwise('UNDEFINED').alias('telephone_interview_acceptable'), \
                when(mapped_df['interview_type'] == lit("0"), lit("NONINTERVIEW/INDETERMINATE")). \
                when(mapped_df['interview_type'] == lit("1"), lit("PERSONAL")). \
                when(mapped_df['interview_type'] == lit("2"), lit("TELEPHONE")).\
                 otherwise('UNDEFINED').alias('interview_type'), \
              when(mapped_df['family_income_range'] == lit("1"), lit("LESS THAN $5,000")). \
                when(mapped_df['family_income_range'] == lit("2"), lit("5,000 TO 7,499")). \
                when(mapped_df['family_income_range'] == lit("3"), lit("7,500 TO 9,999")). \
                when(mapped_df['family_income_range'] == lit("4"), lit("10,000 TO 12,499")). \
                when(mapped_df['family_income_range'] == lit("5"), lit("12,500 TO 14,999")). \
                when(mapped_df['family_income_range'] == lit("6"), lit("15,000 TO 19,999")). \
                when(mapped_df['family_income_range'] == lit("7"), lit("20,000 TO 24,999")). \
                when(mapped_df['family_income_range'] == lit("8"), lit("25,000 TO 29,999")). \
                when(mapped_df['family_income_range'] == lit("9"), lit("30,000 TO 34,999")). \
                when(mapped_df['family_income_range'] == lit("10"), lit("35,000 TO 39,999")). \
                when(mapped_df['family_income_range'] == lit("11"), lit("40,000 TO 49,999")). \
                when(mapped_df['family_income_range'] == lit("12"), lit("50,000 TO 59,999")). \
                when(mapped_df['family_income_range'] == lit("13"), lit("60,000 TO 74,999")). \
                when(mapped_df['family_income_range'] == lit("14"), lit("75,000 TO 99,999")). \
                when(mapped_df['family_income_range'] == lit("15"), lit("100,000 TO 149,999")). \
                when(mapped_df['family_income_range'] == lit("16"), lit("150,000 OR MORE")). \
                 otherwise('UNDEFINED').alias('family_income_range'), \
                when(mapped_df['geo_division'] == lit("1"), lit("NEW ENGLAND")). \
                when(mapped_df['geo_division'] == lit("2"), lit("MIDDLE ATLANTIC")). \
                when(mapped_df['geo_division'] == lit("3"), lit("EAST NORTH CENTRAL")). \
                when(mapped_df['geo_division'] == lit("4"), lit("WEST NORTH CENTRAL")). \
                when(mapped_df['geo_division'] == lit("5"), lit("SOUTH ATLANTIC")). \
                when(mapped_df['geo_division'] == lit("6"), lit("EAST SOUTH CENTRAL")). \
                when(mapped_df['geo_division'] == lit("7"), lit("WEST SOUTH CENTRAL")). \
                when(mapped_df['geo_division'] == lit("8"), lit("MOUNTAIN")). \
                when(mapped_df['geo_division'] == lit("9"), lit("PACIFIC")). \
                otherwise('UNDEFINED').alias('geo_division'), \
                when(mapped_df['race'] == lit("1"), lit("White Only")). \
                when(mapped_df['race'] == lit("2"), lit("Black Only")). \
                when(mapped_df['race'] == lit("3"), lit("American Indian, Alaskan Native Only")). \
                when(mapped_df['race'] == lit("4"), lit("Asian Only")). \
                when(mapped_df['race'] == lit("5"), lit("Hawaiian/Pacific Islander Only")). \
                when(mapped_df['race'] == lit("6"), lit("White-Black")). \
                when(mapped_df['race'] == lit("7"), lit("White-AI")). \
                when(mapped_df['race'] == lit("8"), lit("White-Asian")). \
                when(mapped_df['race'] == lit("9"), lit("White-HP")). \
                when(mapped_df['race'] == lit("10"), lit("Black-AI")). \
                when(mapped_df['race'] == lit("11"), lit("Black-Asian")). \
                when(mapped_df['race'] == lit("12"), lit("Black-HP")). \
                when(mapped_df['race'] == lit("13"), lit("AI-Asian")). \
                when(mapped_df['race'] == lit("14"), lit("AI-HP")). \
                when(mapped_df['race'] == lit("15"), lit("Asian-HP")). \
                when(mapped_df['race'] == lit("16"), lit("W-B-AI")). \
                when(mapped_df['race'] == lit("17"), lit("W-B-A")). \
                when(mapped_df['race'] == lit("18"), lit("W-B-HP")). \
                when(mapped_df['race'] == lit("19"), lit("W-AI-A")). \
                when(mapped_df['race'] == lit("20"), lit("W-AI-HP")). \
                when(mapped_df['race'] == lit("21"), lit("W-A-HP")). \
                when(mapped_df['race'] == lit("22"), lit("B-AI-A")). \
                when(mapped_df['race'] == lit("23"), lit("W-B-AI-A")). \
                when(mapped_df['race'] == lit("24"), lit("W-AI-A-HP")). \
                when(mapped_df['race'] == lit("25"), lit("Other 3 Race Combinations")). \
                when(mapped_df['race'] == lit("26"), lit("Other 4 and 5 Race Combinations")). \
                 otherwise('UNDEFINED').alias('race'))

In [23]:
final_mapped_df.show()

+-----------------+-----------------+----------------+--------------------+--------------------+---------+----------------------+------------------------------+--------------+-------------------+------------------+----------+
|full_household_id|time_of_interview|   final_outcome|   housing_unit_type|      household_type|telephone|access_phone_elsewhere|telephone_interview_acceptable|interview_type|family_income_range|      geo_division|      race|
+-----------------+-----------------+----------------+--------------------+--------------------+---------+----------------------+------------------------------+--------------+-------------------+------------------+----------+
|  000004795110719|          2017/12|   CAPI COMPLETE|HOUSE, APARTMENT,...|HUSBAND/WIFE PRIM...|      YES|                   YES|                           YES|     TELEPHONE|   30,000 TO 34,999|EAST SOUTH CENTRAL|White Only|
|  000004795110719|          2017/12|   CAPI COMPLETE|HOUSE, APARTMENT,...|HUSBAND/WIFE PRIM...|

**Question 1: Count of responders per family income range:**

In [24]:
income_count = final_mapped_df.groupBy("family_income_range").count()
income_count.show()

+-------------------+-----+
|family_income_range|count|
+-------------------+-----+
|   20,000 TO 24,999| 6312|
|   60,000 TO 74,999|13442|
|     7,500 TO 9,999| 2277|
|   30,000 TO 34,999| 6743|
|   75,000 TO 99,999|16557|
|   15,000 TO 19,999| 4518|
|   35,000 TO 39,999| 6620|
|    150,000 OR MORE|15704|
|   40,000 TO 49,999| 9788|
|   12,500 TO 14,999| 2614|
|   LESS THAN $5,000|23527|
|   50,000 TO 59,999| 9971|
|     5,000 TO 7,499| 1625|
|   25,000 TO 29,999| 5803|
| 100,000 TO 149,999|17794|
|   10,000 TO 12,499| 3161|
+-------------------+-----+



**Question 2: Count of responders per geographical division/location and race (top 10):**

In [25]:
geo_race_count = final_mapped_df.groupBy("geo_division", "race").count().orderBy("count", ascending=False)
geo_race_count.show(10)

+------------------+----------+-----+
|      geo_division|      race|count|
+------------------+----------+-----+
|    SOUTH ATLANTIC|White Only|16999|
|          MOUNTAIN|White Only|14343|
|           PACIFIC|White Only|13214|
|EAST NORTH CENTRAL|White Only|11325|
|WEST SOUTH CENTRAL|White Only|11248|
|WEST NORTH CENTRAL|White Only| 9884|
|   MIDDLE ATLANTIC|White Only| 8487|
|       NEW ENGLAND|White Only| 8410|
|EAST SOUTH CENTRAL|White Only| 6580|
|    SOUTH ATLANTIC|Black Only| 4899|
+------------------+----------+-----+
only showing top 10 rows



**Question 3: Responders without a telephone but can access one elsewhere and accept a telephone interview:**

In [26]:
count_telephone = final_mapped_df.filter((mapped_df["telephone"] == "NO") &
                                    (mapped_df["access_phone_elsewhere"] == "YES") &
                                    (mapped_df["telephone_interview_acceptable"] == "YES")).count()
print(f"Count of responders without a telephone but can access one: {count_telephone}")

Count of responders without a telephone but can access one: 0


**Question 4: Responders who can access a telephone, but telephone interview is not accepted:**

In [27]:
count_no_interview = final_mapped_df.filter((mapped_df["telephone"] == "YES") &
                                      (mapped_df["telephone_interview_acceptable"] == "NO")).count()
print(f"Count of responders who can access a telephone but do not accept interviews: {count_no_interview}")

Count of responders who can access a telephone but do not accept interviews: 0


In [28]:
 tel_count = final_mapped_df.groupBy("access_phone_elsewhere").count()
 tel_count.show()

+----------------------+------+
|access_phone_elsewhere| count|
+----------------------+------+
|                  NULL|   917|
|                   YES|140084|
|                    NO|  5455|
+----------------------+------+

